<a href="https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
    !pip install ninja
else:
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !pip install ninja

In [ ]:
!python train.py --help

In [ ]:
!python train.py --gpus=1 --cfg='paper1024' --metrics=None --outdir=./results --data=/content/drive/MyDrive/colab-sg2-ada3/jp-stencil.zip --snap=1 --resume=ffhq1024 --augpipe=bg